### **D2APR: Aprendizado de Máquina e Reconhecimento de Padrões** (IFSP, Campinas) <br/>
**Prof. Dr.**: Samuel Martins (Samuka) <br/>

<a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-nc-sa/4.0/88x31.png" /></a><br />This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/">Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International License</a>. <br/><br/>

### Custom CSS style

In [ ]:
%%html
<style>
.dashed-box {
    border: 1px dashed black !important;
}
.dashed-box tr {
  background-color: white !important;  
}
.alt-tab {
    background-color: black;
    color: #ffc351;
    padding: 4px;
    font-size: 1em;
    font-weight: bold;
    font-family: monospace;
}
// add your CSS styling here
</style>

<span style='font-size: 2.5em'><b>California Housing 🏡</b></span><br/>
<span style='font-size: 1.5em'>Predict the median housing price in California districts</span>

<span style="background-color: #ffc351; padding: 4px; font-size: 1em;"><b>Sprint 6</b></span>

<img src="./imgs/california-flag.png" width=300/>

---



## Before starting this notebook
This jupyter notebook is designed for **experimental and teaching purposes**. <br/>
Although it is (relatively) well organized, it aims at solving the _target problem_ by evaluating (and documenting) _different solutions_ for somes steps of the **machine learning pipeline** — see the ***Machine Learning Project Checklist by xavecoding***. <br/>
We tried to make this notebook as literally a _notebook_. Thus, it contains notes, drafts, comments, etc.<br/>

For teaching purposes, some parts of the notebook may be _overcommented_. Moreover, to simulate a real development scenario, we will divide our solution and experiments into **"sprints"** in which each sprint has some goals (e.g., perform _feature selection_, train more ML models, ...). <br/>
The **sprint goal** will be stated at the beginning of the notebook.

A ***final notebook*** (or any other kind of presentation) that compiles and summarizes all sprints — the target problem, solutions, and findings — should be created later.

#### Conventions

<ul>
    <li>💡 indicates a tip. </li>
    <li> ⚠️ indicates a warning message. </li>
    <li><span class='alt-tab'>alt tab</span> indicates and an extra content (<i>e.g.</i>, slides) to explain a given concept.</li>
</ul>

---

## 🎯 Sprint Goals
- Fine-tune the _hyperparameters_ of the Polynomial Regression models form Sprint #5.
---

### 0. Imports and default settings for plotting

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme(style="whitegrid")

params = {'legend.fontsize': 'x-large',
          'figure.figsize': (15, 5),
         'axes.labelsize': 'x-large',
         'axes.titlesize':'x-large',
         'xtick.labelsize':'x-large',
         'ytick.labelsize':'x-large'}
plt.rcParams.update(params)

## 🛠️ 5. Prepare the Data

We will consider the same two scenarios for **Polynomial Regression** from Sprint #5 in this sprint:
1. Use _only_ the `median_income`.
2. Use _all features_ except those that generated the aggregate features (`total_rooms`, `total_bedrooms`, `population`, `household`).

### 5.1. Load the cleaned training set

Let's consider the training and testing sets already cleaned (sprint #2):
- Drop duplicated instances (no found)
- Drop instances with `housing_median_age` capped at 52
- Drop instances with `median_house_value` capped at 500001.0

In [ ]:
# load the cleaned training set
housing_train = pd.read_csv('./datasets/housing_train_sprint-2.csv')

In [ ]:
housing_train.head()

In [ ]:
housing_train.shape

### 5.2. Separate the _features_ and the _target outcome_

In [ ]:
# store the target outcome into a numpy array
y_train = housing_train['median_house_value'].values

# overwrite the dataframe with only the features  
housing_train = housing_train.drop(columns=['median_house_value'])

### 5.3. Pipelines

For the sake of simplicity, let's include the **Polynomial Regression** objects (`PolynomialFeatures()` + `LinearRegression()`) into our **pipeline**. So, it is no longer _just_ dedicated to preprocessing.

#### **Scenario 1**

In [ ]:
# pipeline for numerical
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression

attributes_scenario_1 = ['median_income']

pipeline_scenario_1 = Pipeline([
    ('imputer', SimpleImputer()),   # let's evaluate the mean and median inputation
    ('poly', PolynomialFeatures()), 
    ('scaler', RobustScaler())
])

# we will just use the ColumnTransformer because it automaticaly filters the required columns for us before performing the pipeline.
# (name, transformer, columns)
preprocessed_pipeline_scenario_1 = ColumnTransformer([
    ("numerical", pipeline_scenario_1, attributes_scenario_1)
])


# full pipeline: preprocessing + model training/prediction
full_pipeline_scenario_1 = Pipeline([
        ('preprocessing', preprocessed_pipeline_scenario_1),
        ('lin_regression', LinearRegression())
])

#### **Scenario 2**

In [ ]:
#### feature engineering method from the Sprint #4
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin

# our 3 new features are based on some the features: totalrooms, 
# column index
rooms_col_idx, bedrooms_col_idx, population_col_idx, households_col_idx = 3, 4, 5, 6

class HousingFeatEngineering(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    
    def fit(self, X, y=None):
        return self  # nothing else to do
    
    def transform(self, X):
        n_rows = X.shape[0]
        
        # creating the new features
        rooms_per_household = X[:, rooms_col_idx] / X[:, households_col_idx]
        population_per_household = X[:, population_col_idx] / X[:, households_col_idx]
        bedrooms_per_room = X[:, bedrooms_col_idx] / X[:, rooms_col_idx]
                
        # to concatenate the new array as columns in our feature matrix, we need to reshape them first
        rooms_per_household = rooms_per_household.reshape((n_rows, 1))
        population_per_household = population_per_household.reshape((n_rows, 1))
        bedrooms_per_room = bedrooms_per_room.reshape((n_rows, 1))
        
        # concatenating the new features into the feature matrix
        X_out = np.hstack((X, rooms_per_household, population_per_household, bedrooms_per_room))
        
        return X_out

In [ ]:
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin

# our 3 new features are based on some the features: totalrooms, 
# column index
rooms_col_idx, bedrooms_col_idx, population_col_idx, households_col_idx = 3, 4, 5, 6

class DropFeatures(BaseEstimator, TransformerMixin):
    def __init__(self, verbose=False):
        self.verbose = verbose
    
    def fit(self, X, y=None):
        return self  # nothing else to do
    
    def transform(self, X):
        X_out = np.delete(X, [rooms_col_idx, bedrooms_col_idx, population_col_idx, households_col_idx], axis=1)
        
        # for debugging
        if self.verbose:
            np.set_printoptions(suppress=True)
            print('X[:5]')
            print(X[:5])
            print('\nX_out[:5]')
            print(X_out[:5])
        
        return X_out

In [ ]:
# pipeline for numerical
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression

attributes_scenario_2 = housing_train.columns.drop('ocean_proximity')

pipeline_scenario_2 = Pipeline([
    ('imputer', SimpleImputer()),  # let's evaluate the mean and median inputation
    ('feat_engineering', HousingFeatEngineering()),
    ('drop_features', DropFeatures(verbose=False)),
    ('poly', PolynomialFeatures()),
    ('scaler', RobustScaler())
])

# we will just use the ColumnTransformer because it automaticaly filters the required columns for us before performing the pipeline.
# (name, transformer, columns)
preprocessed_pipeline_scenario_2 = ColumnTransformer([
    ("numerical", pipeline_scenario_2, attributes_scenario_2)
])

# full pipeline: preprocessing + model training/prediction
full_pipeline_scenario_2 = Pipeline([
        ('preprocessing', preprocessed_pipeline_scenario_2),
        ('lin_regression', LinearRegression())
])

## 🏋️‍♀️ 6. Train ML Algorithms

### 6.1. Grid-Search (fine-tunning)
https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html

### **Scenario 1**

#### **Finding out the hyperparameter key names**

#### **Grid-search**

In [ ]:
cvres = grid_search_scenario_1.cv_results_

for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)

In [ ]:
# best params


In [ ]:
# best estimator
# if refit=True, it returns a model trained with the full training set with the best hyperparameters


In [ ]:
# get the (approximated) RMSE


In [ ]:
# get the approximated RMSE and its standard deviation
best_index = grid_search_scenario_1.best_index_
best_score = np.sqrt(-grid_search_scenario_1.cv_results_['mean_test_score'][best_index])
best_score_std = np.sqrt(grid_search_scenario_1.cv_results_['std_test_score'][best_index])

print(f'Best score: {best_score} +- {best_score_std}')

<br/>

The **cross validation score** from **the best model** (\\$71,298.70 ± \\$11,036.55) is _slightly better_ than the score of Scenario 1 without fine-tunning (\\$71,671.86 ± \$1,479.30) from Sprint #5, its results are more unstable (see its _standard deviation_). <br/>
Moreover, its **cross validation score** keeps being _considerably higher_ than that of `Linear Regression` from the previous Sprint (\\$58,371.04 ± \$1,757.91).

<table align="left" class="dashed-box">
<tr>
    <td>💡</td>
    <td>Mathematically, the correct way to estimate the <b>cross-validation RMSE</b> and its <b>standard deviation</b> is to apply the <i>square root</i> for the each split <i>negative MSE score</i>, and then compute the <i>mean</i> and <i>std</i> with these new scores.</td>
</tr>
<tr>
    <td></td>
    <td>For that, we need to 'debug' the _cross validation results.</td>
</tr>
</table><br/><br/>

In [ ]:
grid_search_scenario_1.cv_results_

In [ ]:
n_folds = 5
split_keys = [f'split{i}_test_score' for i in range(n_folds)]
best_index = grid_search_scenario_1.best_index_

rmse_scores = []

for key in split_keys:
    neg_mse_score = grid_search_scenario_1.cv_results_[key][best_index]
    rmse_scores.append(np.sqrt(-neg_mse_score))

best_rmse = np.mean(rmse_scores)
best_rmse_std = np.std(rmse_scores)


print(f'Best RMSE score: {best_rmse} +- {best_rmse_std}')

By doing it in the _mathematically correct way_, we see that the **true mean RMSE** (\\$71,293.59) is very close from the approximated one (\\$71,298.70). <br/>
However, the **true standard deviation for RMSE** (\$854.29) is _considerably smaller_ than the approximated one (\\$11,036.55).

### **Scenario 2**

In [ ]:
full_pipeline_scenario_2.get_params()

Since our feature matrix has a considerable number of features, for computation time reasons, we decided to:
- Consider a small number of hyperparameter combinations (small search space)
- Low polynomial degrees

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid_scenario_2 = [
    {
     'preprocessing__numerical__imputer__strategy': ['mean', 'median'],
     'preprocessing__numerical__poly__degree': [2, 3, 4, 5],
     'preprocessing__numerical__poly__include_bias': [False, True],
     'preprocessing__numerical__poly__interaction_only': [False]
    }
]

grid_search_scenario_2 = GridSearchCV(full_pipeline_scenario_2, param_grid_scenario_2, cv=5, scoring='neg_mean_squared_error', return_train_score=True, verbose=1)
grid_search_scenario_2.fit(housing_train, y_train)

In [ ]:
cvres = grid_search_scenario_2.cv_results_

for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)

In [ ]:
grid_search_scenario_2.best_params_

In [ ]:
# if refit=True, it returns a model trained with the full training set with the best hyperparameters
best_model = grid_search_scenario_2.best_estimator_
best_model

In [ ]:
# best approximated RMSE
np.sqrt(-grid_search_scenario_2.best_score_)

In [ ]:
# get the approximated RMSE and its standard deviation
best_index = grid_search_scenario_2.best_index_
best_score = np.sqrt(-grid_search_scenario_2.cv_results_['mean_test_score'][best_index])
best_score_std = np.sqrt(grid_search_scenario_2.cv_results_['std_test_score'][best_index])

print(f'Best score: {best_score} +- {best_score_std}')

In [ ]:
n_folds = 5
split_keys = [f'split{i}_test_score' for i in range(n_folds)]
best_index = grid_search_scenario_2.best_index_

rmse_scores = []

for key in split_keys:
    neg_mse_score = grid_search_scenario_2.cv_results_[key][best_index]
    rmse_scores.append(np.sqrt(-neg_mse_score))

best_rmse = np.mean(rmse_scores)
best_rmse_std = np.std(rmse_scores)


print(f'Best RMSE score: {best_rmse} +- {best_rmse_std}')

<br/>

The large **best true RMSE** (\\$345,172.71) -- even larger for the approximated one (\\$626,165.23) -- shows that (i) this model is not suitable for the problem, and/or (ii) the considered _search space_ is insufficient to get better models. <br/>

We'd better We'd better throw this model away.